In [ ]:
"""
Very rough adaptation of Jordan's original python example. Extracts a generic function to read the binary data.

@author: Jordan Raykov
This code replicates the MATLAB example_view_ppp_imu.m and all of the dependent functions. 
The script loads IMU (gyro and accelerometer) sensor data, as well as, json meta-data which is stored in jsonobj  
"""

import os 
import glob
import json
import pandas as pd
from datetime import date
import numpy as np
import matplotlib
import numpy as np
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from matplotlib import mlab

In [ ]:
dtype_mapping = {
    'float': 'f',
    'int': 'i'
}

def read_binary_file(file_path, type, nbits, endianness, ncolumns):
    s_endianness = '<' if endianness == 'little' else '>'
    s_type = dtype_mapping[type]
    s_nbytes = str(nbits // 8)
    format_string = ''.join([s_endianness, s_type, s_nbytes])

    with open(file_path, 'rb') as fid:
        values = np.fromfile(fid, dtype=format_string)
        if ncolumns > 1:
            values = values.reshape((-1, ncolumns))

    return values

In [ ]:
# tsdb load function uses metafilename as input to locate the corresponding 
# path of a json file and sensor data file from specified file_name 
def tsdb_load(metafilename):
    pathstr = os.path.dirname(os.path.abspath(metafilename))
    # Opening JSON file
    f = open(metafilename)
    # returns JSON object as a dictionary
    jsonobj = json.load(f)
    N = len(jsonobj['datasets'])
    unix_ticks_ms = 1000.0;    
    values_list = []    
    data = pd.DataFrame()
    
    start_time_unix = []
    end_time_unix = []
    startsecs = []
    endsecs = []
    device_id = []
    project_id = []
    startdatenum = []
    enddatenum = []
    filename = []
    quantity = []
    units = []
    scale_factors = []
    bits = []
    datatype = []
    rows = []
    cols = []

    for n in range(N):
        ds = jsonobj['datasets'][n]
        start_time_unix.append(float(jsonobj['start_datetime_unix_ms']))
        end_time_unix.append(jsonobj['end_datetime_unix_ms'])
        startsecs = jsonobj['start_datetime_unix_ms']/unix_ticks_ms
        endsecs   = jsonobj['end_datetime_unix_ms']/unix_ticks_ms
        device_id.append(jsonobj['device_id'])
        project_id.append(jsonobj['project_id'])
    
        filename.append(ds['file_name'])
        quantity.append(ds['quantities'])
        units.append(ds['units'])
        scale_factors.append(ds['scale_factors'])
        bits.append(ds['bits'])
        datatype.append(ds['datatype'])
        rows.append(ds['rows'])
        cols.append(ds['columns'])

    
        filenamestr = os.path.join(pathstr, ds['file_name'])
        values = read_binary_file(filenamestr, ds['datatype'], ds['bits'], jsonobj['endianness'], ds['columns'])
        print(values.shape)
        
        values_list.append(values)
    data['start_time_unix_ms'] = start_time_unix
    data['end_time_unix_ms'] = end_time_unix
    data['startsecs'] = startsecs
    data['endsecs'] = endsecs
    data['device_id'] = device_id
    data['project_id'] = project_id
    data['filename'] = filename
    data['quantities'] = quantity
    data['units'] = units
    data['scale_factors'] = scale_factors
    data['bits'] = bits
    data['datatype'] = datatype
    data['rows'] = rows
    data['columns'] = cols
    
    return data, values_list, jsonobj


In [ ]:
# Collect meta data file names

ppp_data_path = '../../data/PPP_IMU_wk10_000077_tsdf';

# Example of loading and plotting all IMU data for a particular user
#ppp_pep_userid = 'FCCE5BD27F7C2F7260169F99B1144FC53CBB8120799A3230FDCC9A68CB549C50'

unix_ticks_ms = 1000.0

# Load each segment's TSDB file in turn, dumping values and time indices
# into a local copy for plotting

meta_segments_list = glob.glob(os.path.join(ppp_data_path, 'WatchData.IMU.Week10.raw_segment*_meta.json'))

meta_filenames = []
for name in meta_segments_list:
    meta_filenames.append(os.path.basename(name))
    
Nfiles = len(meta_filenames)
print(meta_filenames)

In [ ]:
import sys
print(sys.byteorder)

In [ ]:
t = []
v = []

ts = np.zeros((Nfiles, 2))
for n in range(Nfiles):
    meta_fullpath = os.path.join(ppp_data_path, meta_filenames[n])
    data, data_values, jsonobj = tsdb_load(meta_fullpath)
    ts[n] = data['start_time_unix_ms']
    t.append(np.cumsum(np.double(data_values[0])) + ts[n,0])
    v.append(data_values[1])
    del data # For memory conservation, don't keep original data hanging around
    del data_values

fig, axs = plt.subplots(Nfiles)
for n in range(Nfiles):
    tr = (t[n]-ts[n,0])/unix_ticks_ms
    axs[n].plot(tr, v[n]) 
    plt.xlabel('Elapsed time (s)')
    plt.ylabel('IMU')

In [ ]:
dt = np.dtype('>f4')
print(dt, dt.byteorder, dt.itemsize, dt.descr, dt.kind)
dt = np.dtype('float32')
print(dt, dt.byteorder, dt.itemsize, dt.descr, dt.kind)

In [ ]:
# tsdb load function uses metafilename as input to locate the corresponding 
# path of a json file and sensor data file from specified file_name 
def tsdb_load_old(metafilename):
    pathstr = os.path.dirname(os.path.abspath(metafilename))
    # Opening JSON file
    f = open(metafilename)
    # returns JSON object as a dictionary
    jsonobj = json.load(f)
    N = len(jsonobj['datasets'])
    unix_ticks_ms = 1000.0;    
    values_list = []    
    data = pd.DataFrame()
    
    start_time_unix = []
    end_time_unix = []
    startsecs = []
    endsecs = []
    device_id = []
    project_id = []
    startdatenum = []
    enddatenum = []
    filename = []
    quantity = []
    units = []
    scale_factors = []
    bits = []
    datatype = []
    rows = []
    cols = []

    for n in range(N):
        ds = jsonobj['datasets'][n]
        start_time_unix.append(float(jsonobj['start_datetime_unix_ms']))
        end_time_unix.append(jsonobj['end_datetime_unix_ms'])
        startsecs = jsonobj['start_datetime_unix_ms']/unix_ticks_ms
        endsecs   = jsonobj['end_datetime_unix_ms']/unix_ticks_ms
        device_id.append(jsonobj['device_id'])
        project_id.append(jsonobj['project_id'])
    
        filename.append(ds['file_name'])
        quantity.append(ds['quantities'])
        units.append(ds['units'])
        scale_factors.append(ds['scale_factors'])
        bits.append(ds['bits'])
        datatype.append(ds['datatype'])
        rows.append(ds['rows'])
        cols.append(ds['columns'])

    
        filenamestr = os.path.join(pathstr, ds['file_name'])
        # Prepare data type string for binary fread
        if ds['datatype']  == 'double':
            format_var = 'float'+str(ds['bits'])
        else:
            format_var = ds['datatype'] + str(ds['bits'])
            
        #  Decipher byteorder
        if jsonobj['endianness'] == 'little':
            byteorder = '<f4'
        else:
            byteorder = '>f8'
       
        # specify format of the loaded variables, either [Inf 6] for IMU 
        # sensor data or [Inf 1] for timestamp array
        if n == 0: 
            with open(filenamestr, 'rb') as fid:
                values = np.fromfile(fid, dtype=format_var)
        elif n == 1:
            with open(filenamestr, 'rb') as fid:
                values = np.fromfile(fid, dtype=format_var).reshape((-1, 6))
        if len(values)>0:
            values.T

        print(values.shape)

        values_list.append(values)
    data['start_time_unix_ms'] = start_time_unix
    data['end_time_unix_ms'] = end_time_unix
    data['startsecs'] = startsecs
    data['endsecs'] = endsecs
    data['device_id'] = device_id
    data['project_id'] = project_id
    data['filename'] = filename
    data['quantities'] = quantity
    data['units'] = units
    data['scale_factors'] = scale_factors
    data['bits'] = bits
    data['datatype'] = datatype
    data['rows'] = rows
    data['columns'] = cols
    
    return data, values_list, jsonobj